In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import holoviews as hv
hv.extension('bokeh')
import math
from scipy import stats

# SciKit-Learn packages
from sklearn.metrics import precision_score, accuracy_score, hamming_loss, mean_squared_error
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.preprocessing import PolynomialFeatures
# SciKit-multi-learn
from skmultilearn.problem_transform import BinaryRelevance



# Stop words for NLP
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

# Custom functions used in Q2
from Q2Funcs import find_uniques, one_hot, column_score

# Movie Revenue Predictions
Onderzoeksvragen:
1. In hoeverre is de omzet van een film te voorspellen op basis van de populariteit op Facebook en IMDB zelf?
2. In hoeverre is het mogelijk om op basis van plot keywords te voorspellen tot welke genres een film behoort?
3. In hoeverre is het mogelijk om de budget-winst verhouding te voorspellen?

## Het Data Science proces
Voor de eerste verkenning is ons gevraagd om de eerste vier stappen uit te voeren:
1. Data collection
2. Data processing (ook wel data munging)
3. Data cleaning
4. Data exploration & analysis
5. Model building
6. Visualization
7. Communication

## 1. Data Collection
De Data Collection is deels al voor ons gedaan. De dataset `movie.csv` is ons aangeleverd. Echter word voor de opdracht 
gevraagd om deze te combineren met een dataset van derden. Om erachter te komen welke dataset geschikt is om te 
combineren met `movies.csv`zullen wij deze dataset eerst moeten processen, cleanen en exploren.  

Om te zien of de dataset `movie.csv` goed is ingeladen, worden de eerste vijf rijen getoond:

In [ ]:
df_movies = pd.read_csv('data/movie.csv')
df_movies.head()

### Externe dataset
Onze originele dataset bevat geen data over de schrijvers van deze films. Deze data valt echter wel te reconstrueren met
behulp van andere IMDB datasets. Het gaat hierbij om de volgende datasets:

In [ ]:
df_genres = pd.read_csv('data/imdb/movies_genres.csv', delimiter='\t')
df_genres.head()

## 2. Data Processing
### 2a. Aangeleverde dataset
Ook deze stap is grotendeels voor ons gedaan. De data is goed opgeslagen in een `.csv`-bestand en kan direct worden 
ingelezen in een _Pandas_ DataFrame.

Verder rest ons nog de volgende vier stappen:
1. Het verwijderen van ongewenste kolommen
2. Het aanpassen van onduidelijke kolomnamen
3. Het aanpassen van de volgorde van de kolommen
4. Het aanpassen van enkele datatypen

In [ ]:
# Ongewenste kolommen verwijderen
df_movies.drop(["movie_imdb_link", "aspect_ratio"], axis=1, inplace=True)

#Onduidelijke kolomnamen aanpassen
df_movies.rename(columns={'color': 'Colour',
                          'director_name': 'Director',
                          'num_critic_for_reviews': 'Number of critics',
                          'duration': 'Duration',
                          'director_facebook_likes': 'Director FB likes',
                          'actor_3_facebook_likes': 'Actor 3 FB likes',
                          'actor_2_name': 'Actor 2 name',
                          'actor_1_facebook_likes': 'Actor 1 FB likes',
                          'gross': 'Gross',
                          'genres': 'Genres',
                          'actor_1_name': 'Actor 1 name',
                          'movie_title': 'Movie title',
                          'num_voted_users': 'Number of voted users',
                          'cast_total_facebook_likes': 'Total Cast FB likes',
                          'actor_3_name': 'Actor 3 name',
                          'facenumber_in_poster': 'Number of faces on poster',
                          'plot_keywords': 'Plot Keywords',
                          'num_user_for_reviews': 'Number of user reviews',
                          'language': 'Language',
                          'country': 'Country',
                          'content_rating': 'Age rating',
                          'budget': 'Budget',
                          'title_year': 'Release year',
                          'actor_2_facebook_likes': 'Actor 2 FB likes',
                          'imdb_score': 'IMDB Score',
                          'movie_facebook_likes': 'Movie FB likes'}, inplace=True)

# Volgorde kolommen aanpassen
df_movies = df_movies[['Movie title',
                       'Release year',
                       'Director',
                       'Director FB likes',
                       'Gross',
                       'Budget',
                       'Duration',
                       'Language',
                       'Country',
                       'Colour',
                       'Genres',
                       'IMDB Score',
                       'Number of voted users',
                       'Number of critics',
                       'Number of user reviews',
                       'Age rating',
                       'Total Cast FB likes',
                       'Movie FB likes',
                       'Actor 1 name',
                       'Actor 2 name',
                       'Actor 3 name',
                       'Actor 1 FB likes',
                       'Actor 2 FB likes',
                       'Actor 3 FB likes',
                       'Plot Keywords',
                       'Number of faces on poster',
                       ]]

# Datatypen aanpassen
# 1. Floats omzetten naar integers
#  De dataset bevat geen kolommen die dienen te worden bewaard als float, behalve `IMDB Score`
df_movies_IMDB_Score = df_movies["IMDB Score"]  # Tijdelijke kopie van de kolom `IMDB Score`
df_movies = df_movies.drop('IMDB Score', axis=1).fillna(0).astype(int, errors='ignore') # Waarden omzetten naar integers
df_movies.insert(11, "IMDB Score", df_movies_IMDB_Score)  # `IMDB Score` weer toevoegen aan originele DataFrame
del df_movies_IMDB_Score

# 2. De kolom `Release year` omzettten van integers naar het datetime-datatype
df_movies["Release year"] = pd.to_datetime(df_movies["Release year"], format='%Y', errors='coerce')

Het DataFrame ziet er nu als volgt uit:

In [ ]:
df_movies.head()

## 3. Data Cleaning
### 3a. Aangeleverde dataset
1. Het verwijderen van NaN-types
2. Het verwijderen van dubbele filmtitles
3. Het verwijderen van negatieve waardes

In [ ]:
# NaN-types verwijderen
df_movies.dropna(inplace=True)

# Dubbele titels verwijderen
df_movies.sort_values("Release year", inplace=True)  # Sorteren op uitgavejaar
df_movies.drop_duplicates(subset="Movie title", keep="last", inplace=True)  # Alleen meest recente versie blijft bewaard

# Negatieve waardes verwijderen
num = df_movies._get_numeric_data()
num[num < 0] = 0

Na stap 3. Data Cleaning ziet het DataFrame er als volgt uit:

## 4. Data Exploration & Analysis

Nu de data geprepareerd is kunnen wij onze eerste verkenning gaan uitvoeren. Door middel van describe krijgen we
in een oogopslag een duidelijk beeld van het DataFrame `df_movies`. 

In [ ]:
df_movies.describe(include = "all")

Enkele verwachtingen:
1. Een film heeft waarschijnlijk meer Facebook likes dan de director.
2. De meeste films zullen winst maken
3. Het meerendeel van de films zullen in het Engels zijn
4. Comedie en Actie zullen waarschijnlijk de meest voorkomende genres zijn.

Testen van de verwachtingen:
1. Gemiddeld heeft een film meer dan 10x zoveel likes dan een director.
2. Gemiddeld brengt een 40 miljoen op en is er een budget van 30 miljoen voor. Echter heeft de opbrengst een
standaard deviatie van 65 miljoen en het budget een standaard deviatie van 79 miljoen. Een goede conclusie is nog niet te geven.
3. 93.3% van de films zijn in het Engels. Dit is een ruime meerderheid.
4. Tegen de verwachtingen in is `Drama` het meest voorkomende genre met 229 maal. Echter valt hier ook nog niks over 
te zeggen omdat veel films meerdere genres bevatten.

Enkele overige observaties:
- Het DataFrame bevat nog 4811 (95.4%) van de originele 5043 rijen 
- `Movie title` bevat zoals beoogd alleen maar nog unieke waarden.
- De gemiddelde IMDB Score van een film is 6.5 met een standaard deviatie van 1.1. De laagste en hoogste scores zijn 1.6 resp. 9.3.
- Het DataFrame bevat films die uitgegeven zijn tussen 1916 en 2016. Een interval van 100 jaar.
- `Actor 3` bestaat uit de meest (3450) verschillende acteurs, gevolgd door `Actor 2 (2962)`en als laatste `Actor 1 (2042)`.

De tien meest voorkomende Directors en Actor 1's zijn:

In [ ]:
df_movies["Director"].value_counts()[:10]

In [ ]:
df_movies["Actor 1 name"].value_counts()[:10]

De volgende plots zijn slechts voor de eerste analyse van de data. Uiteraard zullen deze nog uitgebreid en verbeterd worden
in de loop van het project. In de volgende scatterplot zijn het budget en de opbrengsten van films geplot. Te zien is dat 
de meeste films niet meer dan 200 miljoen hebben gekost en niet meer dan 400 miljoen opleveren. Enkele uitschieters daargelaten.

In [ ]:
fig, ax = plt.subplots()
plt.title("Relatie tussen het budget en de opbrengst van een film.")
plt.xlabel("Budget")
plt.ylabel("Opbrengst")
plt.xlim(0, 1_000_000_000)
plt.ylim(0, 1_000_000_000)
plt.xticks(np.arange(0, 1_000_000_000, step=100_000_000))
scatter = ax.scatter("Budget", "Gross", data=df_movies)

In de volgende grafiek worden de IMDB scores in het verloop van de tijd geplot. Te zien is dat tot ongeveer 1960 de meeste
films een cijfer tussen de 6 en 9 scoorden. Vanaf 1980 ontstaan er steeds meer films met lagere IMDB score.

In [ ]:
fig, ax = plt.subplots()
plt.title("IMDB Score van films in de loop van de tijd")
plt.xlabel("Jaar van uitgave")
plt.ylabel("IMDB Score")
plt.ylim(0, 10)
plt.yticks(np.arange(0, 10, step=1))
scatter = ax.scatter("Release year", "IMDB Score", data=df_movies)


In de volgende twee boxplots is de spreiding te zien van het budget en de opbrengsten van alle films. Te zien is dat 
het budget van films relatef minder is verspreid dan de opbrengst van de films. De boxplot van de opbrengsten heeft 
tevens veel meer uitschieters.

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2)
plt.suptitle("Boxplots van het budget en de opbrengst van films")
ax1.set_xlabel("Budget")
ax2.set_xlabel("Opbrengst")
ax1.boxplot(df_movies["Budget"])
ax2.boxplot(df_movies["Gross"])
plt.show()

 # Individuele onderzoeksvragen


## Q1. Omzet voorspellen op basis van de populariteit
De onderzoeksvraag gaat als volgt.

> In hoeverre is de omzet van een film te voorspellen op basis van de populariteit op Facebook en IMDB zelf?

Bij deze deelvraag is besloten om de volgorde van de datascience process niet aan te houden. Dat past beter bij deze deelvraag.

### Q1. Data collection

Voor deze onderzoeksvragen wordt er gebruik gemaakt van de volgende features uit `df_movies`:

In [ ]:
# Alleen facebook kolommen of IMDB kolommen nodig
features = ['Director FB likes', 'Actor 1 FB likes', 
                'Actor 2 FB likes', 'Gross', 'Total Cast FB likes',
                'Actor 3 FB likes', 'IMDB Score', 'Movie FB likes']
df_Q1 = df_movies.copy()[features]

### Q1. Data Cleaning

Een gedeelte hiervan hebben we in ```3. Data Cleaning``` gedaan. 

Vervolgens moet alle films zonder omzet verwijderd worden

In [ ]:
df_Q1 = df_Q1[df_Q1.Gross != 0]

### Q1. Data Exploration & Analysis

In [ ]:
df_Q1.describe()

Er zitten grote verschillen tussen waardes van elke kolom. Aangezien machine learning algoritmes algebraïsche zijn betekend dat een hoog getal meer invloed heeft dan een laag getal. Om dit te voorkomen moet de waardes genormaliseerd worden.

Volgende stap is proberen (lineaire) correlaties te vinden.

In [ ]:
plt.matshow(df_Q1.corr())
plt.show()
df_Q1.corr()

Wat hier uit zichtbaar wordt is:
* Acteur 1 heeft veel invloed op total cast likes
* Acteur 2 heeft aanzienlijk minder invloed dan acteur 1 op de total cast likes
* Movie likes heeft zwak invloed op de omzet


### Q1. Data processing

Vervolgens is de dataset opgesplitst naar een input(X) en output(y) waar ze vervolgens gesplitst worden naar een train, test en validatie set.

In [ ]:
# Split the data to X and y
X = df_Q1.drop(columns=['Gross'])
y = df_Q1['Gross']
assert len(X) == len(y) # make sure that the length of x is the same as the length of y



In [ ]:
# Create Train test and validation Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
# Doing it twice because you need a validation set as well
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=1)




### Q1. Model building

Aangezien er zwakke correlatie tussen de input en output is, is een regressie mogelijk.



In [ ]:
LR = LogisticRegression(random_state=1, solver="lbfgs", multi_class="auto", max_iter=200)

LR.fit(X_train, y_train)



Zoals je ziet geeft hij een Convergence warning en kan het zijn dat deze data niet geschikt is voor een logstic regression model

De volgende stap is de waarde van de voorspelling te bepalen.


In [ ]:
mse = mean_squared_error(y_test, LR.predict(X_test))
rms = math.sqrt(abs(mse))
score = LR.score(X_test, y_test)

print("RMS: {}".format(rms))
print("MSE: {}".format(abs(mse)))
print("Score: {:.4f}".format(score))



Na het bekijken van de scores zie je dat dit een erg slecht model is om dit te voorspellen.


#### Linear Regression

Nu we weten dat een logistic regression model niet werkt.

In [ ]:
LR = LinearRegression()

LR.fit(X_train, y_train)


In [ ]:
mse = mean_squared_error(y_test, LR.predict(X_test))
rms = math.sqrt(mse)
score = LR.score(X_test, y_test)

print("RMS: {}".format(rms))
print("MSE: {}".format(mse))
print("Score: {}".format(score))



Nu er een score is zien je dat dit model het beter doet dan ons logistic regression model

### Q1. Data processing

Nadat er een base score is van dit model kan je normalisatie toevoegen en kijken wat de invloed hier op is.

In [ ]:
x_scaled = stats.zscore(X)
y_scaled = stats.zscore(y)

Nadat we de X data gescaled hebben door middel van een zscores moeten we dit weer op splitsen naar train, test en validatie sets.

In [ ]:
# Create Train test and validation Split
X_train, X_test, y_train, y_test = train_test_split(x_scaled, y_scaled, test_size=0.2, random_state=1)
# Doing it twice because you need a validation set as well
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=1)


In [ ]:
LR = LinearRegression()

LR.fit(X_train, y_train)

In [ ]:
mse = mean_squared_error(y_test, LR.predict(X_test))
rms = math.sqrt(mse)
score = LR.score(X_test, y_test)

print("RMS: {}".format(rms))
print("MSE: {}".format(mse))
print("Score: {}".format(score))



Wat we hier bevinden:
* De score is zo goed als gelijk gebleven op vergelijking van de vorige voorspelling met een lineaire regressie
* Mean squared error is in vergelijking stevig naar beneden gegaan

In [ ]:
x = np.linspace(0, 100, len(y_test))
plt.plot(x,y_test, LR.predict(X_test)[:100])


#### Polynomial regression
Nu is een polynomial regression optie.


In [ ]:
polynomial_features= PolynomialFeatures(degree=2)
x_poly = polynomial_features.fit_transform(x_scaled)

In [ ]:
# Create Train test and validation Split
X_train, X_test, y_train, y_test = train_test_split(x_poly, y_scaled, test_size=0.2, random_state=1)
# Doing it twice because you need a validation set as well
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=1)

In [ ]:
LR = LinearRegression()

LR.fit(X_train, y_train)



In [ ]:
mse = mean_squared_error(y_test, LR.predict(X_test))
rms = math.sqrt(mse)
score = LR.score(X_test, y_test)

print("RMS: {}".format(rms))
print("MSE: {}".format(mse))
print("Score: {}".format(score))




Dit levert ons een slechter model op. Zo is de MSE omhoog gegaan en de Score naar beneden

### Q1. Communication
#### Q1. Definitieve pipeline
Onze definitieve model is een lineaire regressie model waarbij de X en y data gescaled is door middel van z-scores


In [ ]:
y = df_Q1['Gross']
y_scaled = stats.zscore(y)

In [ ]:
# Create Train test and validation Split
X_train, X_test, y_train, y_test = train_test_split(x_scaled, y_scaled, test_size=0.2, random_state=1)
# Doing it twice because you need a validation set as well
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=1)

In [ ]:
LR = LinearRegression()

LR.fit(X_train, y_train)

#### Test score

In [ ]:
mse = mean_squared_error(y_test, LR.predict(X_test))
rms = math.sqrt(mse)
score = LR.score(X_test, y_test)

print("RMS: {}".format(rms))
print("MSE: {}".format(mse))
print("Score: {}".format(score))



#### Validation score

In [ ]:
mse = mean_squared_error(y_val, LR.predict(X_val))
rms = math.sqrt(mse)
score = LR.score(X_val, y_val)

print("RMS: {}".format(rms))
print("MSE: {}".format(mse))
print("Score: {}".format(score))
print("Std: {}".format( df_Q1.Gross.std()))

In [ ]:
x = np.linspace(0, len(y_val), len(y_val))
plt.plot(x,y_val, LR.predict(X_val))

### Q2. Conclusie

De onderzoeksvraag die beantwoord wordt met behulp van dit model ging als volgt:
```
In hoeverre is de omzet van een film te voorspellen op basis van de populariteit op Facebook en IMDB zelf?
```

Na het onderzoek is gebleken dat het niet gelukt is door middel van een regressie model een betrouwbare voorspelling te maken. Als je ziet hierboven is het gelukt om een model te maken die 19% van de tijd een goede voorspelling is. Dit valt onder verwachting.



## Q2. Genres voorspellen gebaseerd op Plot Keywords
De onderzoeksvraag gaat als volgt: 
> In hoeverre is het mogelijk om op basis van plot keywords te voorspellen tot welke genres een film behoort? 


### Q2. Data collection
Bij `Q2` wordt er gebruik gemaakt van de kolommen `Genres` en `Plot Keywords` uit `df_movies`. Om ervoor te zorgen dat bij andere onderzoeksvragen de DataFrame niet onnodig wordt aangepast, wordt de dataframe voor de zekerheid als een kopie aangeroepen. 

Ook wordt het model aan het einde van het hoofdstuk getest in een andere scenario. Er wordt getest vanuit de plot beschrijving (I.E. een alinea lange beschrijving van het plot) in plaats van plot keywords. Deze data wordt tegelijkertijd met `DF_Q2` verwerkt tot een X en een Y.

Deze extra data komt van [Document Classification](http://www.davidsbatista.net/blog/2017/04/01/document_classification/).

In [ ]:
df_Q2 = df_movies.loc[:, ("Genres", "Plot Keywords")].copy()
df_Q2.head()

### Q2. Data processing
Aangezien vrijwel alle machine learning algoritmen alleen algebraïsche datatypes accepteren, moeten zowel `Plot Keywords` als `Genres` ingrijpend veranderd worden. Beide zijn namelijk categoriale datatypes.


#### Q2. Cleaning voor processing


Na een korte inspectie van de data, kom je een aantal waardes van 0 en “0” tegen (`int` en `string`). Dit zijn missende waarden. 

Hoewel dit gewoonlijks in stap 3 van het Data Science proces gebeurt, is het in deze scenario beter om dit van tevoren te doen. De 0 en "0" zijn na deze stap namelijk moeilijker terug te vinden.



In [ ]:
# All '0' are missing values, remove
df_Q2.replace("0", np.NaN, inplace=True)
df_Q2.replace(0, np.NaN, inplace=True)
df_Q2.dropna(inplace=True)

#### Q2. Terug naar processing
Om in de gebruikte modellen gebruik te maken van `Genres` en `Plot Keywords` moeten deze gecodeerd worden. 

Om dit te doen moeten de kolommen eerst gesplitst worden. Dit gaat op twee manieren. 

Van `Genres` wordt een lijst gemaakt, alle genres zijn namelijk opgebroken met behulp van een `|`. Dit resulteerd in de kolom `Split Genres`. Hetzelfde proces wordt toegepast bij `Plot Keywords`. Dit resulteerd in de kolom `Split Keywords`.

Bij `Plot Keywords` worden alle `|` vervangen met een spatie. Dit wordt later verder toegelicht.

Hoewel `Split Keywords` niet wordt gebruikt als input variabele, is deze alsnog handig om te hebben bij het analyseren van de data. 

In [ ]:
# Split "Genres" by "|", creating list types
df_Q2.loc[:, "Split Genres"] = df_Q2["Genres"].str.split(pat="|")

# Replace "|" with " " to create strings analysable by TF-IDf.
df_Q2.loc[:, "Plot Keywords"] = df_Q2["Plot Keywords"].str.replace("|", " ")
# Split "Plot Keywords" by " ", creating list types
df_Q2.loc[:, "Split Keywords"] = df_Q2["Plot Keywords"].str.split(pat=" ")


df_Q2.head()

In [ ]:
pd.DataFrame([df_Q2["Split Genres"].str.len().describe(), df_Q2["Split Keywords"].str.len().describe()])

Er zijn gemiddeld drie genres en vijf plot keywords per film. Het is echter géén uniforme data. Zowel genres als plot keywords heeft een minimum van 1 woord (I.E. er is maar één genre of één plot keyword). Het maximaal aantal waarden voor genres is 8, het maximaal aantal waarden voor plot keywords is daarentegen 25. 


Een probleem met de vraagstelling is dat genres multilabel waardes zijn, een film dus heeft meer dan een genre. Alleen het eerste genre pakken is een mogelijkheid, maar dit zal de accuratesse van ons model zeer negatief beïnvloeden. De genres zijn namelijk niet gesorteerd op toepasbaarheid, maar alfabetisch. 

In [ ]:
alphabetical = True
for value in df_Q2["Split Genres"].values:
    if value != sorted(value):
        alphabetical = False
        
alphabetical
    

Zowel genres als plot keywords is categorische data. Aangezien meeste machine learning modellen algebraïsch zijn, accepteren deze alleen numerieke data. Dat betekent dat zowel genres als keywords omgezet moet worden naar numerieke data. 

Dit wordt meestal doormiddel van ‘One-Hot’ encoding gedaan. Bij 'One-Hot' encoding wordt van iedere unieke waarde een eigen kolom gemaakt. Deze kolom heeft een waarde '1' of '0'. Of de waarde komt voor in `Genres`, of niet.

Dit veroorzaakt echter een probleem wanneer er sprake is van een grote variatie aan herhalende data. In deze scenario kan term frequency-inverse document frequency (TF-IDF) gebruikt worden.

TF-IDF genereerd namelijk waardes en kolommen gebaseerd op de frequentie van woorden in een string.

De volgende stap is om te kijken naar unieke waarden van zowel `Genres` als `Plot Keywords`. Gebaseerd op het aantal unieke waarden wordt de de codeertechniek gekozen.

In [ ]:
genre_uniques = find_uniques(df_Q2["Split Genres"])
genre_uniques.describe()

Het gaat hierbij dus om 24 unieke genres. Ook zullen genres niet meer dan één keer in een rij voorkomen. Bij `Genres` zal dus One-Hot encoding gebruikt worden.

In [ ]:
key_uniques = find_uniques(df_Q2["Split Keywords"])
key_uniques.describe()

Er zijn in totaal 6104 unieke plot keywords. Bepaalde plot keywords zullen ook meerdere keren per rij voor kunnen komen. Bij `Plot Keywords` zal dus TF-IDF gebruikt worden.
#### Q2 Coderen van categoriale data
Beginnend met `Genres`, deze kolom wordt gecodeerd met behulp van One-Hot encoding.

In [ ]:
df_Q2 = one_hot(df_Q2, "Split Genres")
Y = df_Q2.loc[:, "Drama" : ]
Y.shape

Ten tweede coderen we `Plot Keywords`, deze kolom wordt gecodeerd met behulp van TF-IDF. 
TF-IDF kan ook veel gebruikte, maar weinig zeggende woorden (`stop words`) als ‘of’ en ‘and’ uit een tekst halen. Deze worden direct door de TF-IDF-vectorizer van Scikit-learn geëxtraheerd. Hierdoor neemt het aantal kolommen significant af.


In [ ]:
vec = TfidfVectorizer(stop_words=stop_words)
X = vec.fit_transform(df_Q2["Plot Keywords"])
X.shape


Hoewel 5994 kolommen nog steeds veel is, bespaar je als nog erg veel in vergelijking tot 6104 kolommen.

#### Q2. Processing van extra dataset

De data van genres staat al One-Hot gecodeerd. Er zijn hier echter wel meer genres dan bij de originele dataset. De extra genres moeten dus verwijderd worden.

In [ ]:
Y_genres = df_genres[np.intersect1d(Y.columns.values, df_genres.columns.values)].copy()
Y_genres.columns.size, Y.columns.size

Er mist nu nog één genre in `Y_genres`. Deze zal overal gelijk staan aan 0.

In [ ]:
missing_genre = np.setdiff1d(Y.columns.values, Y_genres.columns.values)
print("Missing genre:", missing_genre[0])
Y_genres.loc[:, missing_genre[0]] = 0
Y_genres = Y_genres[Y.columns.values]
Y_genres.head()

In [ ]:
Y_genres.columns == Y.columns

Vervolgens moet ook de X van `DF_genres` verwerkt worden. Bij de bron van de dataset moet er eerst gefilterd worden op taal. Hier is dat niet nodig, aangezien de TF-IDF vectorizer gefit is op een Engelse dataset.

In [ ]:
X_genres = vec.transform(df_genres["plot"])
X_genres.shape

### Q2. Data cleaning
Al het Data Cleaning is voor deze stap al gedaan, namelijk in de paragrafen `3. Data Cleaning` en `Q2. Cleaning voor processing`.

### Q2. Data Exploration & Analysis

Nu is een goed moment om te kijken naar de X en Y (noteer de hoofdletter bij Y, de output is namelijk ook een matrix). Beginnend met de X.

De X moet wel eerst omgevormd worden naar een NumPy array. X wordt nu namelijk nog opgeslagen in de vorm van een sparse array. Dit om ruimte in het geheugen te besparen.

In [ ]:
pd.DataFrame(X.toarray(), columns=vec.get_feature_names()).describe()

Hier valt al direct een probleem te ontdekken, de waardes gegenereerd door TF-IDF liggen redelijk laag. Dit betekent dat er niet veel van dezelfde woorden langskomen. Dit kán mogelijk de waarde van de voorspellingen negatief beïnvloeden. Hier wordt echter tot zekere hoogte rekening mee gehouden met behulp van het gebruik van `stop_words`.

Vervolgens de Y. Aangezien er een eigen One-Hot functie geschreven is, wordt Y niet opgeslagen in de vorm van een sparse array. Y is in de vorm van een DataFrame.


In [ ]:
Y.describe()

Aangezien alle waarden of `1` of `0` zijn, staat het gemiddelde ook gelijk aan het percentage films dit genre tot behoort. Zowel té hoog als té laag zal waarschijnlijk een slechte invloed hebben op de voorspellingen. 

Aangezien iets meer dan 50% van de films behoren tot het genre `Drama` behoren, zal er waarschijnlijk sprake zijn van veel false positives. 

Ook zijn er een groot aantal genres waar minder dan 5% van de films tot behoren. Aangezien hier relatief weinig data van is, zal het verbanden vinden tussen plot keywords en deze genres aanzienlijk moeilijker zijn. Waarschijnlijk zal er sprake zijn van veel false negatives. 


#### Extra data set
Tenslotte wordt er ook nog gekeken naar het aantal films per genre bij de extra dataset. 

In [ ]:
unique_extra = pd.DataFrame(Y_genres.sum(), columns=["Count"])
unique_extra.reset_index(inplace=True)
unique_extra.columns = ["Split Genres", "Test Count"]

unique_extra = unique_extra.merge(genre_uniques)
unique_extra.columns = ["Split Genres", "Test Count", "Train Count"]
unique_extra.sort_values("Test Count", ascending=False)

### Q2. Model building


Films zoals “[Bad Boys](https://www.imdb.com/title/tt0112442/?ref_=ttkw_kw_tt)” met genres als ‘Action, Comedy, Crime’ zullen dan alleen geclassificeerd worden als ‘Action’. [Plot keywords](https://www.imdb.com/title/tt0112442/keywords?ref_=tt_ql_stry_4) zoals ‘evil man’ en ‘firearm’ komen dan nog goed overeen met het genre, maar plot keywords als ‘buddy movie’ en ‘buddy comedy’ niet.

Om dit op te lossen, moet er gebruik gemaakt worden van ‘multi-label classification’. Dit is niet te verwarren met ‘multi-class classification’. Bij multi-label is er sprake van meerdere labels die tegelijkertijd toepasbaar kunnen zijn, bij multi-class is er altijd maar één klasse(label) toepasbaar.


Er worden voor deze onderzoeksvraag drie modellen toegelicht.

Ieder model wordt op drie manieren getest:
* Accuracy Score per genre (> == beter)
* Precision Score per genre (> == beter)
* Hamming loss op de gehele voorspelling (< == beter)

Tenslotte wordt de hoeveelheid films per genre als extra kolom toegevoegd.

Ten eerste, het opsplitsen in `test` en `train` datasets:

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=42)
print("Train:", X_train.shape, Y_train.shape)
print("Test:", X_test.shape, Y_test.shape)

#### Q2. Imiteren met behulp van Multi-Class predictions
Eén methode om multi-label te implementeren, is om dit niet te doen. In plaats van “echte” multi-label classificatie te gebruiken, imiteer je het met behulp van Multi-Class classificatie. Voor iedere klasse train en test je het model apart. Dit komt echter met een forse performance hit. Ook zorgt dit ervoor dat voor iedere test set opnieuw het model moet laten fitten.

Qua model wordt er hier gebruik gemaakt van een Multinomial Naive Bayesian model, met daar omheen een veel gebruikte Multi-Class wrapper (OneVsRest). Multinomial Naive Bayesian is vrijwel de standaard voor text classificatie, en zal waarschijnlijk het eerste resultaat zijn indien je text classificatie googled.


In [ ]:
NB_pipeline = Pipeline([
                ('clf', OneVsRestClassifier(MultinomialNB(
                    fit_prior=True, class_prior=None))),
            ])

predictions = []
for genre in genre_uniques["Split Genres"]:
    NB_pipeline.fit(X_train, Y_train[genre])
    predictions.append(NB_pipeline.predict(X_test))
   
predictions = np.array(predictions).T

In [ ]:
print("Hamming loss:", hamming_loss(Y_test, predictions))
column_score(Y_test, predictions).merge(genre_uniques, left_on="Category", right_on="Split Genres").drop("Split Genres", axis=1)

De waarde van de voorspelling lijkt beter te worden naarmate het aantal films per genre het gemiddeld aantal films per genre (564, zie Q2. Data processing) bereikt. Zodra het aantal films hier ruim onder begint te komen, voorspelt dit model dat alle films tot die genres behoren.

De Hamming loss van dit model ligt erg laag, op ongeveer 0.11. Dat betekent dat de voorspeling van redelijk goede kwaliteit is.  


#### Q2. Binary Relevance en Naive Bayes classifiers
De tweede methode om multi-label classification toe te passen, heet Binary Relevance. Dit valt beter uit te leggen met behulp van een voorbeeld. Zie daarvoor de volgende paragraaf. Binary Relevance is op moment van schrijven nog niet opgenomen in het officiële Scikit-Learn library. Om dit te kunnen gebruiken is de Scikit-Learn addon ` skmultilearn` benodigd.
##### Multinomial
Hieronder wordt net zoals bij `Q2. Imiteren met behulp van Multi-Class predictions` gebruik gemaakt van Multinomiale Naive Bayesian classificatie. Met de `BinaryRelevance` wrapper van `skmultilearn` er omheen.

In [ ]:
MNB_classifier = BinaryRelevance(MultinomialNB())
MNB_classifier.fit(X_train, Y_train)

In [ ]:
predictions = MNB_classifier.predict(X_test)

In [ ]:
print("Hamming loss:", hamming_loss(Y_test, predictions))
df_MNB_score = column_score(Y_test, predictions).merge(genre_uniques, left_on="Category", right_on="Split Genres").drop("Split Genres", axis=1)
df_MNB_score

De resultaten van Binary Relevance op Multinomial NB zijn exact hetzelfde als de resultaten resultaten van de eerder gebruikte pipeline. 

Binary Relevance valt binnen multi-label machine learning in het groepje `Problem transformation approaches`. Het pakt een multi-label probleem, en splitst deze op in kleinere problemen die op te lossen zijn met behulp van al bestaande technieken. 
Een nadeel hiervan is echter wel dat de verbanden tússen de verschillende labels buiten beschouwing raken. Het resultaat is desalniettemin beter dan verwacht.


##### Gaussian naive bayes

Een tweede model dat regelmatig wordt gebruikt voor het classificeren van tekst, is het Gaussian Naive Bayes model. Dit gaat uit van een normaalverdeling in plaats van een multinomiale verdeling.

Gaussian is meestal een aanzienlijk slechtere methode om tekst mee te classificeren, het wordt bij deze onderzoeksvraag alleen gebruikt om de kwaliteit van Multinomial Naive Bayes beter te laten zien.


In [ ]:
GNB_classifier = BinaryRelevance(GaussianNB())
GNB_classifier.fit(X_train, Y_train)

In [ ]:
predictions = GNB_classifier.predict(X_test)

In [ ]:
print("Hamming loss:", hamming_loss(Y_test, predictions))
df_GNB_score = column_score(Y_test, predictions).merge(genre_uniques, left_on="Category", right_on="Split Genres").drop("Split Genres", axis=1)
df_GNB_score

Zoals verwacht is de voorspelling minder accuraat dan die van Multinomial NB. De voorspelling is echter wel beter dan verwacht. Het is beter in het voorspellen van de waardes die onder het gemiddelde vallen dan Multinomial NB, voor alle andere waarden lijkt het echter slechter te zijn. 

Tenslotte zijn de laatste drie genres (Film-Noir, Short & News) alle drie nog steeds niet mogelijk om te voorspellen.

#### Q2. Model testing; Extra dataset

Tenslotte wordt dezelfde voorspelling nog een keer uitgevoerd op de extra dataset. Hier wordt weer gebruik gemaakt van de Multinomial Naive Bayesian classifier uit `Q2. Multinomial`.


In [ ]:
predictions = MNB_classifier.predict(X_genres)

In [ ]:
print("Hamming loss:", hamming_loss(Y_genres, predictions))
df_extra_score = column_score(Y_genres, predictions).merge(unique_extra, left_on="Category", right_on="Split Genres").drop("Split Genres", axis=1)
df_extra_score

Hoewel de Hamming loss lager is dan bij de andere voorspellingen, is de precision score aanzienlijk slechter. De mindere waarde is op een paar manieren te verklaren.

De grootte van de originele dataset was te klein om op te trainen. Deze externe dataset is completer, maar daar wordt geen rekening mee gehouden bij het TF-IDF-proces. Er zullen waarschijnlijk veel termen voorkomen in de externe dataset, die niet voorkomen in de originele dataset.

De aard van de externe data is anders dan van het origineel. De externe data bestaat uit een uitgebreide beschrijving van het plot. De originele dataset bevat meerdere enkele termen of korte zinnen. Het kan zijn dat ‘keywords’ zodanig anders zijn dan een beschrijving van het plot, dat dat de voorspelling aanzienlijk negatief beïnvloed.

Het lijkt echter wel dat deze voorspelling hetzelfde pattern volgt als de voorspelling op de testdata. De precision wordt langzamerhand beter totdat deze het genre met het gemiddeld aantal films bereikt. Wanneer het onder dit gemiddelde valt, wordt de precision score 0.

### Q2. Visualization
Om een beter beeld te geven van de verbanden tussen het aantal films per genre, en hun accuracy- en precision scores, worden deze samen geplot.

In [ ]:
sc_MNB_ac = hv.Scatter(df_MNB_score[["Count", "Accuracy Score"]]).opts(title="MNB", ylim=(-0.05, 1.05), xlim=(-100, 2500), size=5)
sc_GNB_ac = hv.Scatter(df_GNB_score[["Count", "Accuracy Score"]]).opts(title="GNB", ylim=(-0.05, 1.05), xlim=(-100, 2500), size=5)
sc_extra_ac = hv.Scatter(df_extra_score[["Train Count", "Accuracy Score"]]).opts(title="Extra MNB", ylim=(-0.05, 1.05), xlim=(-100, 2500), size=5)


sc_MNB_pr = hv.Scatter(df_MNB_score[["Count", "Precision Score"]]).opts(title="MNB", ylim=(-0.05, 1.05), xlim=(-100, 2500), size=5)
sc_GNB_pr = hv.Scatter(df_GNB_score[["Count", "Precision Score"]]).opts(title="GNB", ylim=(-0.05, 1.05), xlim=(-100, 2500), size=5)
sc_extra_pr = hv.Scatter(df_extra_score[["Train Count", "Precision Score"]]).opts(title="Extra MNB", ylim=(-0.05, 1.05), xlim=(-100, 2500), size=5)


(sc_MNB_pr + sc_GNB_pr + sc_extra_pr + sc_MNB_ac + sc_GNB_ac + sc_extra_ac).cols(3)

### Q2. Communication
#### Q2. Definitieve pipeline
De definitieve pipeline van dit project is de `MNB_classifier`. De Binary Relevance Multinomial Naive Bayesian classifier. Van de twee classificatie modellen, geeft deze betere voorspellingen.
#### Q2. Conclusie
De onderzoeksvraag die beantwoord wordt met behulp van dit model ging als volgt:
> In hoeverre is het mogelijk om op basis van plot keywords te voorspellen tot welke genres een film behoort?

Het is zeer goed mogelijk om op basis van plot keywords te voorspellen tot welke genres een film behoort. Er zijn echter wel een paar dingen waar, bij dit model, rekening mee gehouden moet worden. 
Ten eerste is het model, met de huidige training data, niet goed in het voorspellen van genres met minder dan 515 datapunten. Dat houdt dat maar 10 van de 24 genres correct voorspeld kunnen worden. 

De meeste films zullen in principe behoren tot die 10 genres, maar dat betekent niet dat er geen films van die genres bestaan. Bij een dataset van circa 5000 films is dit nog geen groot probleem, bij volledigere datasets (denk aan 100K+) zal dit echter wél voor een probleem veroorzaken.

Ten tweede houdt dit model geen rekening met de onderlinge verbanden tussen genres. Een film behorend tot het genre ‘Biography’ zal vaak ook behoren tot het genre ‘Documentary’. 

#### Q2. Mogelijk vervolg onderzoeken

Bij een mogelijk vervolgonderzoek komen een aantal dingen aan bod. Ten eerste zou dit onderzoek opnieuw uitgevoerd moeten worden op een andere, grotere dataset. 

Ten tweede zou er gekeken moeten worden naar meer geavanceerde multi-label classificatie methoden die wél rekening houden met de onderlinge verbanden tussen genres.

Ten derde zou er gekeken moeten worden naar de verschillen tussen een plot beschrijving (zoals van de externe dataset) en plot keywords (van de originele dataset).


#### Q2 Extra leesmateriaal
[TF-IDF](http://www.tfidf.com)

[Understanding Multi-Label classification model and accuracy metrics](https://medium.com/towards-artificial-intelligence/understanding-multi-label-classification-model-and-accuracy-metrics-1b2a8e2648ca)

[Multi Label Text Classification with Scikit-Learn](https://towardsdatascience.com/multi-label-text-classification-with-scikit-learn-30714b7819c5)

[Binary relevance for multi-label learning: an overview](https://link.springer.com/article/10.1007/s11704-017-7031-7)

